## Train_Validation_Split

In [5]:
# Import modules
import os
import random
import shutil


img_source_dir = '../../datasets/test'
img_target_dir = '../../datasets/data' 

train_size = 0.90


if not os.path.exists(img_target_dir):
    os.makedirs(img_target_dir)
else:
    if not os.path.exists(img_target_dir + '/train'):
        os.makedirs(img_target_dir + '/train')
    if not os.path.exists(img_target_dir + '/validation'):
        os.makedirs(img_target_dir + '/validation')

        
subdirs = [subdir for subdir in os.listdir(img_source_dir) if os.path.isdir(os.path.join(img_source_dir, subdir))]


for subdir in subdirs:
    subdir_fullpath = os.path.join(img_source_dir, subdir)
    train_subdir = os.path.join(img_target_dir + '/train', subdir)
    validation_subdir = os.path.join(img_target_dir + '/validation', subdir)
    
    if not os.path.exists(train_subdir):
        os.makedirs(train_subdir)
        
    if not os.path.exists(validation_subdir):
        os.makedirs(validation_subdir)
        
    train_counter = 0
    validation_counter = 0
    
    # Randomly assign an image to train or validation folder
    for filename in os.listdir(subdir_fullpath):
        if filename.endswith(".jpg") or filename.endswith(".png"): 
            fileparts = filename.split('.')
            
            if random.uniform(0, 1) <= train_size:
                shutil.move(os.path.join(subdir_fullpath, filename), os.path.join(train_subdir, str(train_counter) + '.' + fileparts[1]))
                train_counter += 1
            else:
                shutil.move(os.path.join(subdir_fullpath, filename), os.path.join(validation_subdir, str(validation_counter) + '.' + fileparts[1]))
                validation_counter += 1